In [1]:
import pandas as pd
import numpy as np

In [2]:
sharks = pd.read_csv("../input/attacks.csv", engine="python")

In [3]:
# 25723 entries but a lot of null entries (look at non-null count)
sharks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [4]:
# show first 5 rows, explore
sharks.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson�McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [5]:
# drop unuseful columns: case number = date, year unuseful, name unuseful (anonimous), rest columns not of interest
unuseful_cols = ["Case Number", "Year", "Type", "Location", "Name", "Sex ", "Age", "Time"] + sharks.loc[:, "Investigator or Source":"Unnamed: 23"].columns.to_list()
sharks.drop(columns=unuseful_cols, inplace=True)
sharks.columns

Index(['Date', 'Country', 'Area', 'Activity', 'Injury', 'Fatal (Y/N)',
       'Species '],
      dtype='object')

In [6]:
# rename cols and strip cols
sharks.rename(columns={"Fatal (Y/N)": "Fatal"}, inplace=True)
sharks.columns = [s.strip() for s in sharks.columns]

In [7]:
# how many dates are invalid?
sharks["Date"].str.extract("(\d{2}-\w{3}-\d{4})", expand=False).isnull().sum()

20374

In [8]:
sharks["Date"] = sharks["Date"].str.extract("(\d{2}-\w{3}-\d{4})")
sharks.dropna(subset=["Date"], inplace=True)

In [9]:
sharks.index = pd.to_datetime(sharks["Date"], errors="coerce")
sharks.drop(columns="Date", inplace=True)
sharks.sort_index(inplace=True)
sharks.index

DatetimeIndex(['1703-03-26', '1738-04-06', '1742-12-17', '1751-07-27',
               '1753-10-27', '1771-07-12', '1780-08-08', '1783-03-02',
               '1784-12-12', '1785-09-26',
               ...
               '2018-05-26', '2018-05-27', '2018-06-03', '2018-06-03',
               '2018-06-04', '2018-06-08', '2018-06-09', '2018-06-18',
               '2018-06-25',        'NaT'],
              dtype='datetime64[ns]', name='Date', length=5349, freq=None)

In [10]:
sharks.drop(index=pd.NaT, inplace=True)

In [11]:
sharks.head()

,Country,Area,Activity,Injury,Fatal,Species
Date,,,,,,
1703-03-26,BARBADOS,Southwest coast,Swimming,"Hand and foot severely bitten, surgically ampu...",N,NaN
1738-04-06,ITALY,Sicily,Swimming,FATAL,Y,NaN
1742-12-17,NaN,NaN,Swimming,FATAL,Y,NaN
1751-07-27,USA,Massachusetts,Swimming,FATAL,Y,NaN
1753-10-27,JAMAICA,NaN,Fell overboard,FATAL,Y,NaN


In [12]:
# clean country and area columns
sharks.loc[:, "Country":"Area"] = sharks.loc[:, "Country":"Area"].transform(lambda series: series.str.strip().str.title())

In [13]:
sharks["Fatal"].unique()

array(['N', 'Y', nan, 'UNKNOWN', ' N', '2017', 'M'], dtype=object)

In [14]:
sharks["Fatal"].replace({"UNKNOWN": np.nan, " N": "N", "2017": np.nan, "M": "N"}, inplace=True)

In [15]:
sharks["Species"].value_counts()

White shark                                           143
Shark involvement prior to death was not confirmed     97
Shark involvement not confirmed                        81
Invalid                                                81
Shark involvement prior to death unconfirmed           56
                                                     ... 
White shark, 4.2 m [13'9"]                              1
Bull shark, 400-lb                                      1
Tiger shark, 2.1 m [7']                                 1
White shark, 15' to 16'                                 1
1 m shark, possibly whitetip reef shark                 1
Name: Species, Length: 1413, dtype: int64

In [16]:
sharks["Species"] = sharks["Species"].str.extract("(\w{2,15} shark)", expand=False).str.title()
sharks["Species"].value_counts()

White Shark           578
Tiger Shark           229
Bull Shark            156
Nurse Shark            86
Whaler Shark           63
                     ... 
Unidentified Shark      1
Saw Shark               1
Bonnethed Shark         1
Whiptail Shark          1
Hooked Shark            1
Name: Species, Length: 89, dtype: int64

In [17]:
sharks.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5348 entries, 1703-03-26 to 2018-06-25
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Country   5319 non-null   object
 1   Area      5045 non-null   object
 2   Activity  4970 non-null   object
 3   Injury    5332 non-null   object
 4   Fatal     4848 non-null   object
 5   Species   1840 non-null   object
dtypes: object(6)
memory usage: 292.5+ KB


In [18]:
# keep entries with at least 4 non-null values 
sharks.dropna(thresh=4, inplace=True)
print(f"Number of rows after cleaning: {sharks.shape[0]}")

Number of rows after cleaning: 5156


In [19]:
# export csv
sharks.to_csv("../output/sharks_clean.csv")